In [1]:
import codecs

# Change fp to tweets csv of social_spambots_2.csv from http://mib.projects.iit.cnr.it/dataset.html
bots_fp = '/home/david/Desktop/social_spambots_2.csv/tweets.csv'
genuine_fp = '/home/david/Desktop/genuine_accounts.csv/tweets.csv'

# Open csv file and get the tweet part of the csv.
# Strip out newlines and quotes around text.
with codecs.open(bots_fp, 'r', encoding='utf-8', errors='ignore') as bots_file:
    bot_sentences = [x.split(',')[1].strip('\n').strip('"').lower()
                 if len(x.split(',')) > 1 else '' for x in bots_file.readlines()]
bot_sentences = bot_sentences[1:]
with codecs.open(genuine_fp, 'r', encoding='utf-8', errors='ignore') as genuine_file:
    genuine_sentences = [x.split(',')[1].strip('\n').strip('"').lower()
                 if len(x.split(',')) > 1 else '' for x in genuine_file.readlines()]
genuine_sentences = genuine_sentences[1:]

In [33]:
# Tokenize sentences for gensim word2vec.
neg_tokenizes_sentences = [sentence.split(' ') for sentence in bot_sentences][:10000]
pos_tokenizes_sentences = [sentence.split(' ') for sentence in genuine_sentences][:10000]
tokenizes_sentences = [*pos_tokenizes_sentences, *neg_tokenizes_sentences]

In [34]:
from gensim.models import Word2Vec
# Train a word2vec model.
model = Word2Vec(tokenizes_sentences, min_count=1)

In [9]:
model['dad']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.71445334,  0.9957095 ,  4.001918  ,  1.7210017 , -1.2991213 ,
        1.3544114 ,  1.7409986 , -0.04033926,  0.81119835,  1.438239  ,
       -0.18016362,  0.42983004, -0.5449744 ,  0.18974182,  1.5610733 ,
       -2.1466365 ,  2.443897  ,  2.4384747 ,  0.106648  ,  1.1004133 ,
        2.478804  ,  2.7910225 ,  0.8290645 ,  0.23359032, -1.7706387 ,
       -2.3389952 ,  0.665432  , -2.3710754 ,  0.23790821,  1.5861807 ,
        0.3380512 ,  1.9322393 , -1.1609179 , -0.41444728, -3.8351412 ,
        1.4859318 , -2.0803492 , -1.3367958 , -4.2459364 ,  1.1432422 ,
        0.9030808 ,  0.11530294, -1.08933   , -0.25866237,  2.7305374 ,
       -2.035897  ,  0.66940624, -1.4325031 ,  2.4330492 ,  1.8620452 ,
       -1.2232728 ,  1.4352747 , -3.1087627 ,  3.1435516 , -0.62257165,
       -1.4313347 , -1.9818135 ,  0.48869875, -3.147949  , -2.642702  ,
        1.795755  , -0.16225974,  4.2485285 , -0.24063481, -1.0222123 ,
       -0.47801325,  0.96192735,  2.3427272 ,  0.07277054, -1.07

In [10]:
model.most_similar(positive=['dad'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('mum', 0.9414315223693848),
 ('mom', 0.9369586706161499),
 ('sister', 0.916266143321991),
 ('grandma', 0.9097573757171631),
 ('brother', 0.8948274850845337),
 ('wife', 0.866254448890686),
 ('daughter', 0.8657964468002319),
 ('husband', 0.8598841428756714),
 ('cousin', 0.8504741191864014),
 ('nephew', 0.8450021743774414)]

In [35]:
# Tokenize sentences for gensim word2vec.
bot_tokenizes_sentences = [sentence.split(' ') for sentence in bot_sentences]
genuine_tokenizes_sentences = [sentence.split(' ') for sentence in genuine_sentences]
# Filter out tweets longer than 10 words.
# Limit it to the first 10k to work in jupyter notebook
bot_tokenizes_sentences_10 = list(filter(lambda x: len(x) <= 10, bot_tokenizes_sentences))[:10000]
genuine_tokenizes_sentences_10 = list(filter(lambda x: len(x) <= 10, genuine_tokenizes_sentences))[:10000]

In [36]:
def create_training_data(tokenized_sentences, word2vec_model, pos_or_neg):
    '''
    Creates training data by returing the embedded version of the sentence with the label.
    Currently only using tweets of 10 words or less.
    
    :tokenized_sentences: list of list of words
    :word2vec_model: trained word2vec model
    :pos_or_neg: 0 if genuine, 1 if bot
    
    Returns:
    X: list of embedded sentences
    Y: List of Labels
    '''
    pruned_embedded_sentences = []
    for sentence in tokenized_sentences:
        embedded_sentence = list()
        for word in sentence:
            if word not in word2vec_model:
                embedded_sentence = None
                break
            embedded_sentence = [*embedded_sentence, *word2vec_model[word]]
        if embedded_sentence is not None:
            if len(embedded_sentence) != 10*100:
                diff = 10*100 - len(embedded_sentence)
                embedded_sentence += [0]*diff
            pruned_embedded_sentences.append(embedded_sentence) 

    return pruned_embedded_sentences, [pos_or_neg] * len(pruned_embedded_sentences)

In [37]:
pos_x,pos_y = create_training_data(bot_tokenizes_sentences_10, model, 1)
len(pos_x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


5619

In [38]:
neg_x,neg_y = create_training_data(genuine_tokenizes_sentences_10, model, 0)
len(neg_x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


5052

In [39]:
x = [*pos_x, *neg_x]
y = [*pos_y, *neg_y]

In [40]:
# Following https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# First neural network with keras tutorial
from keras.models import Sequential
from keras.layers import Dense

nn_model = Sequential()
nn_model.add(Dense(200, input_dim=1000, activation='relu'))
nn_model.add(Dense(50, activation='relu'))
nn_model.add(Dense(1, activation='sigmoid'))

nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
import numpy as np
nn_model.fit(np.array(x), np.array(y), epochs=100, batch_size=10)

Epoch 1/100
10671/10671 [==============================] - 2s 228us/step - loss: 0.6835 - accuracy: 0.5673
Epoch 2/100
10671/10671 [==============================] - 2s 216us/step - loss: 0.6610 - accuracy: 0.5846
Epoch 3/100
10671/10671 [==============================] - 2s 216us/step - loss: 0.6450 - accuracy: 0.6107
Epoch 4/100
10671/10671 [==============================] - 2s 217us/step - loss: 0.6283 - accuracy: 0.6194
Epoch 5/100
10671/10671 [==============================] - 2s 217us/step - loss: 0.6196 - accuracy: 0.6313
Epoch 6/100
10671/10671 [==============================] - 2s 218us/step - loss: 0.6130 - accuracy: 0.6350
Epoch 7/100
10671/10671 [==============================] - 2s 217us/step - loss: 0.6061 - accuracy: 0.6369
Epoch 8/100
10671/10671 [==============================] - 2s 219us/step - loss: 0.5978 - accuracy: 0.6483
Epoch 9/100
10671/10671 [==============================] - 2s 218us/step - loss: 0.5960 - accuracy: 0.6444
Epoch 10/100
10671/10671 [===========

Epoch 77/100
10671/10671 [==============================] - 2s 222us/step - loss: 0.4078 - accuracy: 0.7894
Epoch 78/100
10671/10671 [==============================] - 2s 224us/step - loss: 0.4024 - accuracy: 0.7972
Epoch 79/100
10671/10671 [==============================] - 2s 216us/step - loss: 0.4032 - accuracy: 0.7958
Epoch 80/100
10671/10671 [==============================] - 2s 216us/step - loss: 0.4041 - accuracy: 0.7967
Epoch 81/100
10671/10671 [==============================] - 2s 220us/step - loss: 0.3930 - accuracy: 0.8004
Epoch 82/100
10671/10671 [==============================] - 2s 221us/step - loss: 0.3909 - accuracy: 0.8031
Epoch 83/100
10671/10671 [==============================] - 2s 221us/step - loss: 0.3930 - accuracy: 0.8029
Epoch 84/100
10671/10671 [==============================] - 3s 237us/step - loss: 0.3969 - accuracy: 0.7936
Epoch 85/100
10671/10671 [==============================] - 2s 230us/step - loss: 0.3932 - accuracy: 0.7995
Epoch 86/100
10671/10671 [==

In [42]:
_, accuracy = nn_model.evaluate(np.array(x), np.array(y))
print('Accuracy: %.2f' % (accuracy*100))

10671/10671 [==============================] - 0s 21us/step
Accuracy: 82.13
